# 2.0 Strings

## 2.1 Find anagram indices

### Problem Statement
Given a word $w$ and string $s$ find the starting indices of all of the anagrams of $s$ in $w$.

anagram (def.)
> An anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.  For example, `adobe` can be rearranged into `abode`.

In [1]:
import collections
import unittest


def anagram_indices(word, s):
    """Find the starting indices of anagrams of s in word."""

    # Incrementally update a character count frequency map as
    # you move a sliding window of length s over word.
    # The count in the map associated with each character
    # has the following invariants:
    # 0  = character appears with same frequency in window
    # <0 = character not in s appears in window
    # >0 = character in s appears with overabundance in window
    wfreq, inds = collections.defaultdict(int), []

    # Initialize the frequency with contents of s.
    for char in s:
        wfreq[char] += 1
        
    # Decrement the frequency with the characters in first window.
    # Instead of explicitly keeping 0-valued entries in the map,
    # delete such entries so that the test for anagrams is changed
    # to be a test where map is empty when window contains anagram.
    for char in word[:len(s)]:
        wfreq[char] -= 1  # Decrement means character in window.
        if wfreq[char] == 0:
            del wfreq[char]
    if len(wfreq) == 0:
        inds.append(0)

    # Iterate from [len(s), len(word)] incrementally updating map.
    for ind in range(len(s), len(word)):
        cleft, cright = word[ind-len(s)], word[ind]
        wfreq[cleft] += 1  # Increment means character not in window.
        if wfreq[cleft] == 0:
            del wfreq[cleft]
        wfreq[cright] -= 1  # Decrement means character in window.
        if wfreq[cright] == 0:
            del wfreq[cright]
        if len(wfreq) == 0:
            inds.append(ind-len(s)+1)

    return inds


class AnagramIndicesTest(unittest.TestCase):

    def test_anagram_indices(self):
        case = collections.namedtuple('case', ['word','s','expected'])
        cases = [
            case('abxaba', 'ab', [0,3,4]),
            # Corner case with repeating elements of s.
            case('abxabb', 'ab', [0,3]),
            # Corner case with repeating elements followed by anagram.
            case('abxabba', 'ab', [0,3,5]),
            # Corner case with back-to-back-to-back.
            case('liveevilvile', 'live', [0,4,8]),
            # Corner case with same characters but different frequency.
            case('ttwwt', 'wwt', [1,2]),
        ]
        for c in cases:
            rcv = anagram_indices(c.word ,c.s)
            self.assertEqual(rcv, c.expected)


unittest.main(AnagramIndicesTest(), argv=[''], verbosity=2, exit=False)

test_anagram_indices (__main__.AnagramIndicesTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## 2.2 Generate palindrome pairs

### Problem Statement
Given a list of words, return all possible palindromes formed by concatenating pairs of words from the list.

palindrome (def.)
> A palindrome is a word, number, phrase, or other sequence of characters which reads the same backward as forward.  For example, the word `madam` is a palindrome.

In [2]:
import collections
import unittest


def is_palindrome(word):
    """Return True when a word is a palindrome."""
    if len(word) < 2:
        return True  # Empty and 1-character strings are palindromes.
    return word == ''.join(reversed(word))


def palindrome_pairs(words):
    """Return palindrome pairs formed by concatenating pairs of words."""
    pairs, lookup = [], set(words)

    for word in words:
        # Check whether the palindrome of the word is in the input list.
        if not is_palindrome(word):
            reversed_word = ''.join(reversed(word))
            if reversed_word in lookup:
                pairs.append((word, reversed_word))

        # Check whether word prefix forms palindrome.
        for ind in range(0,len(word)-1):
            if not(is_palindrome(word[ind+1:])):  # Suffix not palindrome.
                continue
            reversed_word = ''.join(reversed(word[:ind+1]))
            if reversed_word in lookup:
                pairs.append((word, reversed_word))
        
        # Check whether word suffix forms palindrome.
        for ind in range(len(word)-1,0,-1):
            if not(is_palindrome(word[:ind])):  # Prefix not palindrome.
                continue
            reversed_word = ''.join(reversed(word[ind:]))
            if reversed_word in lookup:
                pairs.append((reversed_word, word))

    return pairs


class PalindromePairsTest(unittest.TestCase):

    def test_palindrome_pairs(self):
        case = collections.namedtuple('case', ['input','expected'])                                                                   
        cases = [
            # Palindrome formed without overlapping prefix or suffix.
            case(['foo','oof'], [('foo','oof'),('oof','foo')]),
            # One palindrome pair found by matching prefix to `dc`.
            case(['cdaba','dc'], [('cdaba', 'dc')]),
            case(['cdabc','dc'], []),  # Suffix is not palindrome.
            # One palindrome pair found by matching suffix to `dc`.
            case(['abacd','dc'], [('dc','abacd')]),
            case(['abccd','dc'], []),  # Prefix is not palindrome.
            # Words are palindromes, but no pair form palindromes.
            case(['aba', 'cbc'], []),
            # Fully general case mixing palindromes and non-palindromes.
            case(["ma","dam","nurses","run","foo","oof","fo","f"],
                 [('ma','dam'),
                  ('nurses','run'),
                  ('foo','f'),('foo','oof'),
                  ('oof','foo'),
                  ('fo','f'),('fo','oof'),
                  ('f','oof')]),
        ]
        for c in cases:
            rcv = palindrome_pairs(c.input)
            self.assertEqual(sorted(rcv), sorted(c.expected))


unittest.main(PalindromePairsTest(), argv=[''], verbosity=2, exit=False)

test_palindrome_pairs (__main__.PalindromePairsTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## 2.3 Print a string in zigzag form

### Problem Statement
Implement a function that takes a string, named s, and a height, named h, and prints the string in zigzag form.

#### Examples
input
```
helloworldzigzag
1234567890123456
```

h=2
```
h l o o l z g a
 e l w r d i z g
```

h=4
```
h     o     g
 e   w r   i z
  l o   l z   a
   l     d     g
```

In [3]:
import collections
import io
import unittest


def zigzag(string, height):
    """Return a printed string of given height in zigzag form."""
    strbuff = io.StringIO()
    # Print the string line-by-line into the buffer varying the
    # stride between characters depending upon whether we are 
    # printing characters from top-to-bottom or bottom-to-top.
    for h in range(1,height+1):
        # stride1 refers to top-to-bottom and stride2 bottom-to-top.
        stride1, stride2 = (height*2) - (2*h), 2*h-2
        # Alternating stride isn't necessary for top and bottom.
        stride2 = stride1 if h == 1 else stride2
        stride1 = stride2 if h == height else stride1
        strbuff.write(' '*(h-1))  # Add leading padding.
        # Print each character separated by the stride length.
        ind, stride = h-1, stride1
        while True:
            strbuff.write(string[ind])  # Print character.
            ind += stride
            if not(ind < len(string)):
                break
            strbuff.write(' '*(stride-1))  # Space between characters.
            stride = stride2 if stride == stride1 else stride1  # Toggle.
        strbuff.write('\n')
    return strbuff.getvalue()


class ZigzagTest(unittest.TestCase):

    helloworld = 'helloworldzigzag'
    
    helloworld_h2 = \
"""
h l o o l z g a
 e l w r d i z g
"""

    helloworld_h4 = \
"""
h     o     g
 e   w r   i z
  l o   l z   a
   l     d     g
"""

    helloworld_h5 = \
"""
h       l
 e     r d     g
  l   o   z   a
   l w     i z
    o       g
"""

    def test_zigzag(self):
        case = collections.namedtuple('case', ['s','h','expected'])
        cases = [
            case(self.helloworld, 2, self.helloworld_h2),
            case(self.helloworld, 4, self.helloworld_h4),
            case(self.helloworld, 5, self.helloworld_h5),
        ]
        for c in cases:
            rcv = zigzag(c.s, c.h)
            # NOTE(marcoemorais): Remove leading linebreak from t.q.s.
            expected = c.expected[1:]
            self.assertEqual(rcv, expected)


unittest.main(ZigzagTest(), argv=[''], verbosity=2, exit=False)

test_zigzag (__main__.ZigzagTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## 2.4 Determine smallest rotated string

### Problem Statement
Given a string and length $k$, determine the lexicographically smallest string that can be created by rotating $k$ characters from the beginning of the string to the end.

lexicographic order (def.)
> Generalization of the way words are alphabetically ordered based on the alphabetical order of their component letters.  For example, the lexicographic of the string `ab` comes before `ac`.